### Lorentz System

$\frac{\mathrm{d}x}{\mathrm{d}t} = \sigma (y - x), \\[6pt]$


$\frac{\mathrm{d}y}{\mathrm{d}t} = x (\rho - z) - y, \\[6pt]$


$\frac{\mathrm{d}z}{\mathrm{d}t} = x y - \beta z.$

$σ = 10$, $β = 8/3$ and $ρ = 28$

In [60]:
import autograd.numpy as np
from autograd import grad, elementwise_grad
from matplotlib import pyplot as plt

### Activation Functions

In [51]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def tanh(z):
    return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))


def elu(z, alpha: float):
    return alpha * (np.exp(z) - 1) if z < 0 else z

### Trial Solution & Analytical Solution

In [52]:
def trial_solution(
    input: np.array, parameters: list, initial_conditions: np.array
) -> np.array:
    return np.linspace(
        initial_conditions, initial_conditions, input.size
    ).T + input * forward_propogation(input, parameters)


def right_hand_side(
    input: np.array, trial_sol: np.array, constants: np.array
) -> np.array:
    return np.array(
        [
            constants[0] * (trial_sol[1] - trial_sol[0]),
            trial_sol[0] * (constants[2] - trial_sol[2]) - trial_sol[1],
            trial_sol[0] * trial_sol[1] - constants[1] * trial_sol[2],
        ]
    )


def lorenz(xyz, *, s=10, r=28, b=2.667):
    """
    Parameters
    ----------
    xyz : array-like, shape (3,)
       Point of interest in three-dimensional space.
    s, r, b : float
       Parameters defining the Lorenz attractor.

    Returns
    -------
    xyz_dot : array, shape (3,)
       Values of the Lorenz attractor's partial derivatives at *xyz*.
    """
    x, y, z = xyz
    x_dot = s * (y - x)
    y_dot = r * x - y - x * z
    z_dot = x * y - b * z
    return np.array([x_dot, y_dot, z_dot])

### Loss Function

In [53]:
def MSE_loss_function(
    input, parameters: list, constants: np.array, initial_conditions: np.array
):
    trial_sol = trial_solution(input, parameters, initial_conditions)
    prediction = right_hand_side(input, trial_sol, constants)
    neural_network_gradient = elementwise_grad(trial_solution, 0)(
        input, parameters, initial_conditions
    )

    error_squared = (neural_network_gradient - prediction) ** 2

    loss_sum = np.sum(error_squared)

    return loss_sum / np.shape(error_squared)[0]

### Neural Network

In [46]:
def init_parameters(input_size: int, hidden_sizes: np.array, output_size: int):
    np.random.seed(0)

    # hidden weights and biases
    hidden_weights = [np.random.randn(hidden_sizes[0], input_size)]
    for i in range(1, hidden_sizes.shape[0]):
        hidden_weights.append(np.random.randn(hidden_sizes[i], hidden_sizes[i - 1]))

    hidden_biases = [np.zeros((h, 1)) for h in hidden_sizes]

    # output weights and biases
    output_weights = np.random.randn(output_size, hidden_sizes[-1])
    output_bias = np.zeros((output_size, 1))

    parameters = [hidden_weights, hidden_biases, output_weights, output_bias]
    return parameters

In [47]:
def forward_propogation(input: np.array, parameters: list):
    hidden_weights, hidden_biases, output_weights, output_bias = parameters
    num_hidden_layers = len(hidden_weights)

    # hidden layers
    z = np.matmul(hidden_weights[0], input) + hidden_biases[0]
    a = sigmoid(z)
    for i in range(1, num_hidden_layers):
        z = np.matmul(hidden_weights[i], a) + hidden_biases[i]
        a = sigmoid(z)

    # output layer
    z = np.matmul(output_weights, a) + output_bias
    a = sigmoid(z)
    return a

In [48]:
parameters = init_parameters(1, np.array([2, 3, 4]), 3)
print(parameters[2])

[[-2.55298982  0.6536186   0.8644362  -0.74216502]
 [ 2.26975462 -1.45436567  0.04575852 -0.18718385]
 [ 1.53277921  1.46935877  0.15494743  0.37816252]]


In [49]:
def back_propogation(
    input: np.array,
    parameters: list,
    constants: np.array,
    initial_conditions: np.array,
    num_iter: int,
    learn_rate: float,
):
    loss_grad_function = grad(MSE_loss_function, 1)

    for i in range(num_iter):
        loss_grad = loss_grad_function(input, parameters, constants, initial_conditions)
        # hidden layers
        for j in range(len(parameters)):
            for k in range(len(parameters[j])):
                parameters[j][k] = parameters[j][k] - learn_rate * loss_grad[j][k]

    return parameters

In [73]:
t = np.arange(0, 10, 0.01)
t = np.reshape(t, (1, 1000))



constants = np.array([10.0, 8 / 3, 28.0])
initial_conditions = np.array([0.0, 1.0, 1.05])
num_iter = 5
learn_rate = 0.001



parameters = init_parameters(1, np.array([10]), 3)
print(parameters)
parameters = back_propogation(
    t, parameters, constants, initial_conditions, num_iter, learn_rate
)
print(parameters)


res = trial_solution(t, parameters, initial_conditions)
print(res)

[[array([[ 1.76405235],
       [ 0.40015721],
       [ 0.97873798],
       [ 2.2408932 ],
       [ 1.86755799],
       [-0.97727788],
       [ 0.95008842],
       [-0.15135721],
       [-0.10321885],
       [ 0.4105985 ]])], [array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])], array([[ 0.14404357,  1.45427351,  0.76103773,  0.12167502,  0.44386323,
         0.33367433,  1.49407907, -0.20515826,  0.3130677 , -0.85409574],
       [-2.55298982,  0.6536186 ,  0.8644362 , -0.74216502,  2.26975462,
        -1.45436567,  0.04575852, -0.18718385,  1.53277921,  1.46935877],
       [ 0.15494743,  0.37816252, -0.88778575, -1.98079647, -0.34791215,
         0.15634897,  1.23029068,  1.20237985, -0.38732682, -0.30230275]]), array([[0.],
       [0.],
       [0.]])]
[[array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]])], [array([[nan

In [33]:
np.repeat(initial_conditions, 2, axis=0)

array([0.  , 0.  , 1.  , 1.  , 1.05, 1.05])

In [35]:
np.linspace(initial_conditions, initial_conditions, 1000).T

array([[0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [1.  , 1.  , 1.  , ..., 1.  , 1.  , 1.  ],
       [1.05, 1.05, 1.05, ..., 1.05, 1.05, 1.05]])